In [13]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import mlflow
from sklearn.ensemble import RandomForestClassifier
import sklearn



In [11]:
mlflow.set_tracking_uri("sqlite:///../mlflow.db")
mlflow.set_experiment("titanic")

<Experiment: artifact_location='/media/tomislav/023ABC973ABC8965/Projekti/MLOps-Testing/src/mlruns/2', creation_time=1694604295202, experiment_id='2', last_update_time=1694604295202, lifecycle_stage='active', name='titanic', tags={}>

In [3]:
dataset_path = "../data/titanic/"
train_df = pd.read_csv(os.path.join(dataset_path, "train.csv"))
test_df = pd.read_csv(os.path.join(dataset_path, "test.csv"))


print(train_df.shape)
print(test_df.shape)


(891, 12)
(418, 11)


In [4]:
nullseries = train_df.isnull().sum()
print(nullseries[nullseries > 0])

Age         177
Cabin       687
Embarked      2
dtype: int64


In [5]:
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
from sklearn.metrics import accuracy_score
from mlflow.data.pandas_dataset import PandasDataset
from mlflow.models import infer_signature


mlflow.sklearn.autolog(disable=True)

def preprocess_dataset(df, test=False):
    in_features = ['Pclass', 'Sex', 'SibSp', 'Parch']
    if test:
        return pd.get_dummies(df[in_features]) 
    
    out_features = ['Survived']
    return pd.get_dummies(df[in_features]), df[out_features]

def kCrossVal(k, model_class, model_params, data):
    model = model_class()
    model.set_params(**model_params)

    val_ratio = 1.0 / k
    interval = len(data) * val_ratio
    interval = np.floor(interval).astype(np.int16)

    splits = []
    pool = np.array(range(len(data)))
    for i in range(int(1/val_ratio)):
        split = np.random.choice(pool, size=interval, replace=False)
        split = split.tolist()
        splits.append(split)
        pool = pool[np.isin(pool, split, invert=True)]
    for i in range(len(splits)):
        print(len(splits[i]))

    sum_score = 0.0
    iter_num = len(splits)

    #K-fold Cross-Validation
    for i in range(len(splits)):
        val_pool = splits[i]
        train_pool = []
        for j in range(len(splits)):
            if i == j:
                continue
            train_pool.append(splits[j])
        train_pool = np.hstack(train_pool).tolist()

        train_data = data.iloc(axis=0)[train_pool]
        val_data = data.iloc(axis=0)[val_pool]
        
        X_train, y_train = preprocess_dataset(train_data)
        X_val, y_val = preprocess_dataset(val_data)

        model.fit(X_train, np.ravel(y_train))
        val_pred = model.predict(X_val)
        acc = accuracy_score(y_true=y_val, y_pred=val_pred)
        sum_score += acc
    return sum_score / iter_num

        
with mlflow.start_run(run_name='rf_baseline'):
    mlflow.set_tag("model_name", "RF")

    model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
    param = model.get_params()
    
    acc = kCrossVal(k = 5, 
                    model_class = RandomForestClassifier, 
                    model_params = param, 
                    data = train_df)

    X_train, y_train = preprocess_dataset(train_df)
    model.fit(X_train, np.ravel(y_train))

    joined_train = pd.concat((X_train,y_train),axis=1)
    mlflow_train_dataset: PandasDataset = mlflow.data.from_pandas(joined_train)
    mlflow.log_input(mlflow_train_dataset, context="training")
    mlflow.log_params(params=param)
    mlflow.log_metric("accuracy", acc)
    signature = infer_signature(model_input=X_train, model_output=y_train)
    mlflow.sklearn.log_model(model, "sk_models", signature=signature)

    print(acc)
        

178
178
178
178
178


/home/tomislav/mambaforge-pypy3/envs/titanic_env/lib/python3.11/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/home/tomislav/mambaforge-pypy3/envs/titanic_env/lib/python3.11/site-packages/mlflow/data/pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-inte

0.7966292134831461


In [34]:

X_test = preprocess_dataset(test_df, test=True)

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_df.PassengerId,
                       'Survived': predictions})
output.to_csv('../kaggle/titanic/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
